In [1]:
import pandas as pd


In [2]:
test = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/Test_TP2_Datos_2020-2C.csv')

train = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/Train_TP2_Datos_2020-2C.csv')

In [3]:
def limpiar_DF(df):
    # inicialmente voy con el tratamiento mas simple sobre los datos faltantes
    df = df.fillna(0)
    df = df.dropna()
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # irrelevantes
    df = df.drop(['Opportunity_Name'], axis=1)

    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
        
    return df

In [4]:
test = limpiar_DF(test)
train = limpiar_DF(train)

In [5]:
import category_encoders as ce

In [6]:
#Duda, hay que hacerle encoding binario tambien a las columnas numericas?

def binary_encoding(df):

    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    nuevodf = pd.DataFrame()

    for i in categoric_cols:
        encoder = ce.BinaryEncoder(i)
        df_enco = encoder.fit_transform(df[i])
        nuevodf = pd.concat([nuevodf, df_enco], axis = 1)
    
    nuevodf = pd.concat([nuevodf, df[numeric_cols]], axis = 1)
    
    nuevodf = pd.concat([nuevodf, df[numeric_cols_2]], axis = 1)
    
    return nuevodf



In [7]:
trian_binary = binary_encoding(train)
trian_binary

/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/

,Region_0,Region_1,Region_2,Region_3,Territory_0,Territory_1,Territory_2,Territory_3,Territory_4,Territory_5,...,ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Delivery_Year,TRF,Stage
0,0,0,0,1,0,0,0,0,0,0,...,27761,1,1,1,1,0,0,2016,10,0
1,0,0,0,1,0,0,0,0,0,0,...,27760,0,0,0,0,0,1,2016,0,1
2,0,0,1,0,0,0,0,0,0,0,...,27446,0,0,0,0,0,2,2016,0,1
3,0,0,1,0,0,0,0,0,0,0,...,16808,1,0,1,0,0,3,2018,14,0
4,0,0,1,0,0,0,0,0,0,0,...,16805,1,0,1,0,0,4,2018,25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16942,0,0,0,1,0,0,0,0,1,0,...,8781,1,1,1,1,0,12801,2016,0,1
16943,0,0,0,1,0,0,0,0,1,0,...,8786,1,1,1,1,0,12801,2016,0,1
16944,0,0,0,1,0,0,0,0,1,0,...,8792,1,1,1,1,0,12801,2016,0,1
16945,0,0,1,0,0,0,0,0,1,0,...,28561,1,1,1,1,0,12802,2016,4,0


In [8]:
trian_binary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16883 entries, 0 to 16946
Columns: 234 entries, Region_0 to Stage
dtypes: float64(4), int64(230)
memory usage: 30.3 MB


In [9]:
#guardo un scv con este encoding
trian_binary.to_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/TrainBinaryEncoding.csv')

In [10]:
#lo mismo con el test
test_binary = binary_encoding(test)
test_binary.to_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/TestBinaryEncoding.csv')

/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/leandro/anaconda3/

In [11]:
test_binary.shape

(2551, 209)

In [12]:
trian_binary.shape

(16883, 234)

In [13]:
test_binary

,Region_0,Region_1,Region_2,Region_3,Territory_0,Territory_1,Territory_2,Territory_3,Territory_4,Territory_5,...,Total_Taxable_Amount,ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Delivery_Year,TRF
0,0,0,0,1,0,0,0,0,0,0,...,367419.0,6140,1,1,1,1,0,10689,2019,0
1,0,0,0,1,0,0,0,0,0,0,...,367419.0,6146,1,1,1,1,0,10689,2019,0
2,0,0,0,1,0,0,0,0,0,0,...,367419.0,6151,1,1,1,1,0,10689,2019,0
3,0,0,0,1,0,0,0,0,0,0,...,757783.5,6118,1,1,1,1,0,10690,2019,1
4,0,0,0,1,0,0,0,0,0,0,...,757783.5,6124,1,1,1,1,0,10690,2019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,0,0,0,1,0,0,0,1,0,1,...,147750.0,16345,1,1,0,0,0,12364,2019,0
2547,0,0,1,0,0,0,1,0,0,0,...,45054.9,15218,1,1,0,0,0,12365,2019,0
2548,0,0,1,0,0,0,1,0,0,0,...,100122.0,15224,1,1,1,1,0,12366,2019,0
2549,0,0,1,0,0,0,0,0,0,1,...,143220.0,7286,1,1,0,0,0,12367,2019,0


In [14]:
columnsTrain = trian_binary.columns
columnsTest = test_binary.columns

temp3 = [x for x in columnsTrain if x not in columnsTest]
temp3 = set(temp3) - set(['Stage'])

temp3

#Bueno aca se ve como train tiene columnas extras que generan problemas a la hora de la prediccion (salvo stage)
#voy a poner en test estas columnas con 0


{'ASP_Currency_3',
 'Account_Created_Date_10',
 'Account_Name_10',
 'Account_Name_11',
 'Billing_Country_7',
 'Last_Modified_Date_10',
 'Last_Modified_Date_8',
 'Last_Modified_Date_9',
 'Month_6',
 'Opportunity_Created_Date_10',
 'Opportunity_Created_Date_11',
 'Opportunity_Created_Date_8',
 'Opportunity_Created_Date_9',
 'Opportunity_Type_5',
 'Planned_Delivery_End_Date_10',
 'Planned_Delivery_Start_Date_10',
 'Product_Family_8',
 'Product_Name_9',
 'Quote_Expiry_Date_10',
 'Quote_Expiry_Date_9',
 'Quote_Type_1',
 'Territory_7',
 'Total_Amount_Currency_3',
 'Total_Taxable_Amount_Currency_3'}

In [15]:
extra = pd.DataFrame(columns=temp3)


test_binary = pd.concat([test_binary, extra], axis = 1)
test_binary = test_binary.fillna(0)

test_binary.to_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/TestBinaryEncoding.csv')

In [16]:
test_binary

,Region_0,Region_1,Region_2,Region_3,Territory_0,Territory_1,Territory_2,Territory_3,Territory_4,Territory_5,...,Last_Modified_Date_9,Billing_Country_7,Product_Name_9,Total_Amount_Currency_3,Quote_Expiry_Date_9,Opportunity_Created_Date_8,Account_Name_11,Last_Modified_Date_8,ASP_Currency_3,Product_Family_8
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2547,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2548,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2549,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
